This notebook contains all code to reproduce results in the paper "Analyzing e-cigarette sentiment of Twitter", published in the Computational Health Sciences Workshop at the 6th ACM Conference on Bioinformatics, Computational Biology, and Health Informatics, 2015.

###Data

We begin with the data collected by Sherry Emery et al. <slemery@uic.edu>, which consists 4.6M tweets from 2012-10-01 to 2013-09-30. These tweets have already been classified as "organic" or not using an SVM classifier (see Huang, Jidong et al. "A cross-sectional examination of marketing of electronic cigarettes on Twitter." Tobacco control). We restrict our analysis to those classified as organic. We will assume these data live in `/data/chs15/ecig.csv.gz`.

In [274]:
from collections import Counter, defaultdict
import cPickle
import csv
import datetime
import gzip
import itertools
import json
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import re
import requests
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate

DATA = '/data/chs15'

In [3]:
def read_csv(filename, fields=['text', 'svm', 'hand_label', 'posted_time', 'real_name', 'username']):
    f = gzip.open(filename, 'rb')
    csvr = csv.DictReader(f, delimiter=',', quotechar='"')
    for row in csvr:
        yield dict([(k, row[k]) for k in fields if k in row])

In [5]:
# Read all "organic" tweets.
raw_tweets = [r for r in read_csv(DATA + '/ecig.csv.gz') if r['svm'] == '-']
print('read %d "organic" tweets' % len(raw_tweets))

read 992633 "organic" tweets


Next, we manually labeled 2,000 tweets into one of two categories:
- **positive (1):** express positive sentiment toward ecigs, or indicate that the speaker uses ecigs.
- **negative (0):** express negative sentiment toward ecigs *or* do not express sentiment, e.g., informative

(We originally separated negative into two classes, but accuracy was too low to support further analysis.)

We assume these data live in `/data/chs15/labeled.csv.gz`.

In [258]:
labeled_tweets = [r for r in read_csv(DATA + '/labeled.csv.gz', fields=['text', 'sent', 'real_name', 'username'])]

print('read %d labeled tweets' % len(labeled_tweets))
# Set labels.

label_map = {'-1': 'negative', '0': 'negative', '1': 'positive'}
for t in labeled_tweets:
    t['sent'] = label_map[t['sent']]
             
label_encoder = LabelEncoder()
label_encoder.fit(['negative', 'positive'])
y = label_encoder.transform([t['sent'] for t in labeled_tweets])             
print('Label distribution=%s' % Counter(t['sent'] for t in labeled_tweets).most_common(3))

read 2000 labeled tweets
Label distribution=[('negative', 1296), ('positive', 704)]


### Classifier Training

Using `labeled_tweets`, we'll train a logistic regression classifier.

In [259]:
# Tweet tokenizer.
def tokenize(text):
    punc_re = '[' + re.escape(string.punctuation) + ']'
    text = text.lower()
    text = re.sub('#(\S+)', r'HASHTAG_\1', text)
    text = re.sub('@(\S+)', r'MENTION_\1', text)
    text = re.sub('http\S+', 'THIS_IS_A_URL', text)
    text = re.sub(r'(.)\1\1\1+', r'\1', text)
    text = re.sub(r'[0-9]', '9', text)
    toks = []
    for tok in text.split():
        tok = re.sub(r'^(' + punc_re + '+)', r'\1 ', tok)
        tok = re.sub(r'(' + punc_re + '+)$', r' \1', tok)
        for subtok in tok.split():
            if re.search('\w', subtok):
                toks.append(subtok)
    return toks

In [260]:
vectorizer = TfidfVectorizer(decode_error='ignore', ngram_range=(1, 2), max_df=1., min_df=2,
                             use_idf=True, tokenizer=tokenize, binary=False, norm='l2')
X = vectorizer.fit_transform(t['text'] for t in labeled_tweets)
print('Vectorized %d tweets. Found %d terms.' % (X_labeled.shape[0], X.shape[1]))
features = np.array(vectorizer.get_feature_names())

Vectorized 2000 tweets. Found 4824 terms.


In [264]:
def confusion(truths, preds, labels):
    m = confusion_matrix(truths, preds)
    m = np.vstack((labels, m))
    m = np.hstack((np.matrix([''] + list(labels)).T, m))
    return tabulate(m.tolist(), headers='firstrow')

def top_coef(clf, vocab, n=10):
    if len(clf.classes_) == 2:
        coefs = [clf.coef_[0], -clf.coef_[0]]
    else:
        coefs = clf.coef_
    for li, label in enumerate(clf.classes_):
        print('\nCLASS %s' % label)
        coef = coefs[li]
        top_coef_ind = np.argsort(coef)[::-1][:n]
        top_coef_terms = vocab[top_coef_ind]
        top_coef = coef[top_coef_ind]
        print '\n'.join(['%s\t%.3f' % (term, weight) for term, weight in zip(top_coef_terms, top_coef)])

def do_cv(X, y, labels, nfolds=10):
    cv = KFold(len(y), nfolds, random_state=123456)
    preds = []
    truths = []
    for train, test in cv:
        clf = LogisticRegression(class_weight='auto')
        clf.fit(X[train], y[train])
        preds.extend(clf.predict(X[test]))
        truths.extend(y[test])
    print ('accuracy=%.3f' % (accuracy_score(truths, preds)))
    print classification_report(truths, preds, target_names=labels)
    print confusion(truths, preds, labels)
    clf = LogisticRegression(class_weight='auto')
    clf.fit(X, y)
    return clf, truths, preds

clf, truths, preds = do_cv(X, y, label_encoder.classes_, 10)
top_coef(clf, features, 5)

accuracy=0.793
             precision    recall  f1-score   support

   negative       0.84      0.83      0.84      1296
   positive       0.70      0.72      0.71       704

avg / total       0.79      0.79      0.79      2000

            negative    positive
--------  ----------  ----------
negative        1082         214
positive         199         505

CLASS 0
my	5.390
i	4.668
vaping	2.160
HASHTAG_vaping	1.722
HASHTAG_ecig	1.663

CLASS 1
THIS_IS_A_URL	4.017
e-cigarettes	1.940
de	1.171
as	1.164
99	1.112


In [271]:
# Write cross-validation results .tex table.
def clfreport_to_tex(report, outfile):
    """ Write a sklearn classification report as a latex table. """
    report = re.sub(r' \/ total', '', report)
    report = re.sub(r'precision', 'Prec', report)
    report = re.sub(r'recall', 'Rec', report)
    report = re.sub(r'f1-score', 'F1', report)
    report = re.sub(r'support', 'N', report)
    table = ['\\begin{tabular}{|r|c|c|c|c|}', '\\hline']
    lines = report.split('\n')
    for i, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) > 0:
            if i == 0:
                parts = [''] + ['{\\bf %s}' % p for p in parts]
            else:
                parts[0] = '{\\bf %s}' % parts[0]
            table.append(' & '.join(parts) + '\\\\')
        else:
            table.append('\\hline')
    table.append('\\end{tabular}')
    of = open(outfile, 'wt')
    of.write('\n'.join(table))
    
clfreport_to_tex(classification_report(truths, preds, target_names=labels), 'cv.tex')

In [272]:
# Write top coef .tex table.
def clean(s):
    s = re.sub('HASHTAG_', '\#', s)
    s = re.sub('MENTION_', '@', s)
    s = re.sub('THIS_IS_A_URL', 'URL', s)
    s = re.sub(r'_', '\\_', s)
    return s
    
def write_top_coef(clf, vocab, labels, outf, n=20):
    out = open(outf, 'wt')
    coefs = [clf.coef_[0], -clf.coef_[0]]
    for li, label in enumerate(labels):
        coef = coefs[li]
        top_coef_ind = np.argsort(coef)[::-1][:n]
        top_coef_terms = vocab[top_coef_ind]
        out.write('{\\bf %s} & %s\\\\\n\hline\n' % (label, ', '.join(clean(s) for s in top_coef_terms)))
        
write_top_coef(clf, features, labels, 'coef.tex', n=20)


### Applying classifier

Next, we classify all the unlabeled tweets using the classifier.

In [273]:
X_raw = vectorizer.transform(t['text'] for t in raw_tweets)

In [277]:
preds_raw = clf.predict(X_raw)
print('predicted label distribution on raw tweets: %s' % Counter(preds_raw).most_common(2))
for tweet, pred in zip(raw_tweets, preds_raw):
    t['sent'] = labels[pred]

predicted label distribution on raw tweets: [(0, 623396), (1, 369237)]


### Gender

Next, we classify each user by gender using a list of names from the census.

In [282]:
def get_gender_names(cutoff=75):
    males_url = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.male.first'
    females_url = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.female.first'
    males = set([l.split()[0].lower() for l in requests.get(males_url).text.split('\n') if l and float(l.split()[2]) < cutoff])
    females = set([l.split()[0].lower() for l in requests.get(females_url).text.split('\n') if l and float(l.split()[2]) < cutoff ])
    print('found %d male and %d female names with cutoff=%.2f' % (len(males), len(females), cutoff))
    return remove_ambiguous_names(males, females)

def remove_ambiguous_names(male_names, female_names):
    ambiguous = male_names & female_names
    male_names -= ambiguous
    female_names -= ambiguous
    print('removed %d ambiguous names, leaving %d males and %d females' % (len(ambiguous), len(male_names), len(female_names)))
    return male_names, female_names
    
male_names, female_names = get_gender_names()

found 232 male and 523 female names with cutoff=75.00
removed 6 ambiguous names, leaving 226 males and 517 females


In [285]:
def label_genders(tweets, male_names, female_names):
    for t in tweets:
        if len(t['real_name'])>1:
            first = t['real_name'].split()[0].lower()
        else:
            first = t['real_name'].lower()
        if first in male_names:
            t['gender'] = 'male'
        elif first in female_names:
            t['gender'] = 'female'
        else:
            t['gender'] = 'unknown'
label_genders(raw_tweets, male_names, female_names)
print('overall gender distribution=%s' % Counter(t['gender'] for t in raw_tweets).most_common(3))

overall gender distribution=[('unknown', 732856), ('male', 156902), ('female', 102875)]


In [315]:
print('most common male names: %s' % Counter(t['real_name'].split()[0] for t in raw_tweets if t['gender'] == 'male').most_common(10))
print('most common female names: %s' % Counter(t['real_name'].split()[0] for t in raw_tweets if t['gender'] == 'female').most_common(10))
print('most common unknown names: %s' % Counter(t['real_name'].split()[0] for t in raw_tweets if t['real_name'] and t['gender'] == 'unknown').most_common(10))

most common male names: [('Mark', 9017), ('Paul', 4531), ('John', 4393), ('Michael', 3748), ('Chris', 3662), ('Adam', 3539), ('Alex', 3215), ('Gregory', 2916), ('David', 2747), ('James', 2732)]
most common female names: [('Sarah', 2007), ('Ashley', 1399), ('Jessica', 1320), ('Emily', 1318), ('Rachel', 1212), ('Amanda', 1208), ('Katie', 1121), ('Laura', 1116), ('Mandy', 1087), ('Lauren', 1084)]
most common unknown names: [('Electronic', 15305), ('Dragonfly', 14279), ('The', 5898), ('Lewisville', 4837), ('g', 4588), ('Vapornine', 4105), ('Matt', 3573), ('?', 3237), ('DFW', 2904), ('Nick', 2466)]


### Age

To guess age from a user's name, we use the idea from from [Nate Silver's](https://twitter.com/FiveThirtyEight) recent article on [How to Tell Someone's Age When All you Know is Her Name](http://fivethirtyeight.com/features/how-to-tell-someones-age-when-all-you-know-is-her-name/). We borrow from [this notebook](https://github.com/ramnathv/agebyname_py) by [@ramnathv](https://github.com/ramnathv).

First, we download a list of [baby names](http://www.ssa.gov/oact/babynames/names.zip) provided by the SSA.

In [287]:
import urllib
names_file = DATA + '/names.zip'
urllib.urlretrieve('http://www.ssa.gov/oact/babynames/names.zip', DATA + '/names.zip')

('/data/chs15/names.zip', <httplib.HTTPMessage instance at 0x11ed967e8>)

In [336]:
# Read list of names.
import pandas as pd
from zipfile import ZipFile

def read_names(f, zf):
  data = pd.read_csv(zf.open(f), header = None, names = ['name', 'sex', 'n'])
  data['year'] = int(re.findall(r'\d+', f)[0])
  return data
  
names_zip = ZipFile(names_file)
bnames = pd.concat([read_names(f, names_zip) for f in names_zip.namelist() if f.endswith('.txt')])
names_ =  set(bnames.name)
bnames.head()

,name,sex,n,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880


Next we parse the [cohort life tables](http://www.ssa.gov/oact/NOTES/as120/LifeTables_Tbl_7.html) provided by SSA, using @ramnathv's export.

In [337]:
from scipy.interpolate import interp1d

lifetables = pd.read_csv(DATA + '/lifetables.csv')
lifetables_2014 = lifetables[lifetables['year'] + lifetables['x'] == 2014]
lifetables_2014.head()

# interpolate gaps
def process(d, kind = 'slinear'):
  f = interp1d(d.year, d.lx, kind)
  year = np.arange(1900, 2011)
  lx = f(year)
  return pd.DataFrame({"year": year, "lx": lx, "sex": d.sex.iloc[1]})

lifetable_2014 = lifetables_2014.\
  groupby('sex', as_index = False).\
  apply(process)
lifetable_2014.head()

lx sex  year
0 0   0.0   F  1900
  1  19.1   F  1901
  2  38.2   F  1902
  3  57.3   F  1903
  4  76.4   F  1904

Finally, we use the [live births data](http://www.census.gov/statab/hist/02HS0013.xls) from the census to extrapolate the birth data to account for the correct that not all births were recorded by SSA till around 1930, since it wasn't mandatory.

In [338]:
# Download the data and add a correction factor.
urllib.urlretrieve("http://www.census.gov/statab/hist/02HS0013.xls", DATA + "/02HS0013.xls")
dat = pd.read_excel(DATA + '/02HS0013.xls', sheetname = 'HS-13', skiprows = range(14))
tot_births = dat.ix[9:101,:2].reset_index(drop = True)
tot_births.columns = ['year', 'births']
tot_births = tot_births.convert_objects(convert_numeric = True)
print tot_births.head()
# Correction factors.
cor_factors = bnames.groupby('year', as_index = False).sum().merge(tot_births)
cor_factors['cor'] = cor_factors['births']*1000/cor_factors['n']
cor_factors = cor_factors[['year', 'cor']]
cor_new = pd.DataFrame({
  'year': range(2002, 2014),
  'cor': cor_factors.cor.iloc[-1]
})
cor_factors = pd.concat([cor_factors, cor_new])[['year', 'cor']]
print cor_factors.head()

   year  births
0  1909    2718
1  1910    2777
2  1911    2809
3  1912    2840
4  1913    2869
   year       cor
0  1909  5.316621
1  1910  4.701051
2  1911  4.359994
3  1912  2.874354
4  1913  2.523141


In [339]:
# For efficiency, define name as index of the table
bname= bnames.set_index(['name'], inplace=True)

In [340]:
# This function tells the number of births and number of people alive. 
def get_data(name):
    dat = bnames
    dat = dat.loc[name]
    if type(dat) == pd.Series:
        m = pd.DataFrame(bnames.loc[name]).transpose()
    else:
        m = dat
    data =   m.\
            merge(cor_factors).\
            merge(lifetable_2014)
    data['n_cor'] = data['n']*data['cor']
    data['n_alive'] = data['lx']/(10**5)*data['n_cor']
    return data

print('Joseph')
print get_data('Edward').tail()
print('Irene')
print get_data('Irene').tail()

Joseph
    sex     n  year       cor       lx        n_cor      n_alive
193   F     7  2007  1.076384  99351.7     7.534690     7.485842
194   M  2820  2007  1.076384  99221.6  3035.403577  3011.775996
195   M  2783  2008  1.076384  99256.4  2995.577360  2973.302247
196   M  2979  2009  1.076384  99291.2  3206.548673  3183.820656
197   M  2898  2010  1.076384  99326.0  3119.361549  3098.337052
Irene
    sex    n  year       cor       lx       n_cor     n_alive
169   F  492  2006  1.076384  99324.6  529.581050  526.004259
170   F  467  2007  1.076384  99351.7  502.671443  499.412624
171   F  482  2008  1.076384  99378.8  518.817207  515.594315
172   F  479  2009  1.076384  99405.9  515.588054  512.524946
173   F  408  2010  1.076384  99433.0  439.164773  436.674709


In [341]:
# Now we can map each name to a distribution over age brackets.
def first_name(name):
    if len(name) > 1:
        name = name.split()[0].lower().title()
    else:
        name = name.lower().title()
    return name


def age_brackets(name):    
    name = first_name(name)
    dic = defaultdict(list)
    #
    s=0 
    s1=0
    s2=0
    s3=0
    s4=0
    s5=0
    #fraction of percentages
    f=0  
    f1=0
    f2=0
    f3=0
    f4=0    
    f5=0
    
    m = get_data(name).as_matrix()
    if len(m) != 0:
        for t in m:
            if t[2]>=1996: # 0-18
                s = s+t[1]
                f = f + t[6]
                dic['<18'] = f
            elif t[2] >= 1990 and t[3]<=1995: #19-24
                s1 = s1+t[1]
                f1 = f1 + t[6]
                dic['19-24'] = f1
            elif t[2] >= 1980 and t[3]<=1989: #25-34
                s2 = s2+t[1]
                f2 = f2 + t[6]
                dic['25-34'] = f2
            elif t[2] >= 1970 and t[3]<=1979: #35-44
                s3 = s3+t[1]
                f3 = f3 + t[6]
                dic['35-44'] = f3
            elif t[2] <= 1960 :#and t[2] <= 1969: #45-54
                s4 = s4+t[1]
                f4 = f4 + t[6]
                dic['>45'] = f4
        #d = pd.DataFrame(dic.values(),dic.keys(), columns=['n','n_alive'])
        #d['fraction']= (d['n_alive']/d['n_alive'].sum())*100        
        #print d
        #print dic
        total = sum(dic.values())
        return dict([(k, 1.*v/total*100) for k, v in dic.iteritems()])
    
print('Irene')
print age_brackets('Irene')
print('Michael')
print age_brackets('Michael')
print('Payton')
print age_brackets('Payton')

Irene
{'19-24': 2.768896935891503, '35-44': 5.9900770087280355, '>45': 79.96979901362864, '25-34': 5.300358368654281, '<18': 5.970868673097543}
Michael
{'19-24': 10.25125353909169, '35-44': 21.724002632069023, '>45': 33.05996262251043, '25-34': 21.20556963381044, '<18': 13.759211572518417}
Payton
{'19-24': 6.8700057813423445, '35-44': 0.4265604073936273, '>45': 0.5359894837394966, '25-34': 1.2200035112432313, '<18': 90.9474408162813}


In [343]:
def label_ages(tweets, valid_names):
    name_cache = {}
    for ti, t in enumerate(tweets):
        name = first_name(t['real_name'])
        if name in valid_names:
            if name in name_cache:
                distr = name_cache[name]
            else:
                distr = age_brackets(name)
                name_cache[name] = distr
            t['ages'] = distr
        
        if ti % 10000 == 0:
            print('processed %d tweets' % ti)
        ti += 1
        
label_ages(raw_tweets, names_)

processed 0 tweets
processed 10000 tweets
processed 20000 tweets
processed 30000 tweets
processed 40000 tweets
processed 50000 tweets
processed 60000 tweets
processed 70000 tweets
processed 80000 tweets
processed 90000 tweets
processed 100000 tweets
processed 110000 tweets
processed 120000 tweets
processed 130000 tweets
processed 140000 tweets
processed 150000 tweets
processed 160000 tweets
processed 170000 tweets
processed 180000 tweets
processed 190000 tweets
processed 200000 tweets
processed 210000 tweets
processed 220000 tweets
processed 230000 tweets
processed 240000 tweets
processed 250000 tweets
processed 260000 tweets
processed 270000 tweets
processed 280000 tweets
processed 290000 tweets
processed 300000 tweets
processed 310000 tweets
processed 320000 tweets
processed 330000 tweets
processed 340000 tweets
processed 350000 tweets
processed 360000 tweets
processed 370000 tweets
processed 380000 tweets
processed 390000 tweets
processed 400000 tweets
processed 410000 tweets
proces

In [352]:
def add_to_dict(d, d1):
    for k in d1:
        d[k] += d1[k]
    
all_ages = defaultdict(lambda: 0)
for t in raw_tweets:
    if 'ages' in t and t['ages']:
        add_to_dict(all_ages, t['ages'])
        
print 'overall ages:'
for k, v in all_ages.iteritems():
    print('%s\t%.2f' % (k, v / sum(all_ages.values())))

overall ages:
19-24	0.12
>45	0.27
35-44	0.14
<18	0.30
25-34	0.16


In [ ]:
# Save our annotated tweets.
cPickle.dump(raw_tweets, open(DATA + '/raw_tweets.pkl', 'wb'))

### Figures

Next we create a number of figures to explore how ecig sentiment varies by time, gender, and age.